In [ ]:
import numpy as np 
import pandas as pd 

import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re
import string

import numpy as np

from textblob import TextBlob
#from afinn import Afinn
#from textstat.textstat import textstat

from nltk.corpus import stopwords

#from preprocess import normalize_texttall

from datetime import datetime

In [ ]:
es= pd.read_csv('/kaggle/input/jigsaw-toxic-more-language/train_es.csv')
fr= pd.read_csv('/kaggle/input/jigsaw-toxic-more-language/train_fr.csv')
de= pd.read_csv('/kaggle/input/jigsaw-toxic-more-language/train_de.csv')
en= pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv')

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
es.head()
fr.head()
de.head()

In [ ]:
weights = {}
for c in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    weights[c]=(es[c]==1).sum()

total_weight = sum(weights.values())

for c in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    weights[c]=np.log(total_weight/weights[c])
weights

In [ ]:
#3 different labels: weighted score ys,1-0 class y,sum(total 5 classes) y5
es['ys']=0
for c in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    es['ys']=es['ys']+es[c]*weights[c]
es['y5'] = (es[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)).astype(int)
es['y'] = (es[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0).astype(int)
#df = df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})
es.head(5)
es['lan']='es'
es['ys'].value_counts()

In [ ]:
def label_df(tmp,lan):
    tmp['ys']=0
    for c in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
        tmp['ys']=tmp['ys']+tmp[c]*weights[c]
    tmp['y5'] = (tmp[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1)).astype(int)
    tmp['y'] = (tmp[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0).astype(int)
    #df = df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})
    tmp.head(5)
    tmp['lan']=lan
    tmp['ys'].value_counts()
    return tmp

In [ ]:
fr = label_df(fr,'fr')
de = label_df(de,'de')
en = label_df(en,'en')

# data cleaning

In [ ]:
def print_step(step):
    print('[{}]'.format(datetime.now()) + ' ' + step)

# validation data

In [ ]:
val = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
val.head()

In [ ]:
less = val['less_toxic'].to_frame().rename(columns={'less_toxic':'comment_text'})
more = val['more_toxic'].to_frame().rename(columns={'more_toxic':'comment_text'})

In [ ]:
ft_cols = [c for c in less.columns]
#ft_cols
ft_cols.remove('comment_text')
#ft_cols.remove('clean_text')

In [ ]:
less.to_csv('pro_less.csv')
more.to_csv('pro_more.csv')

# pipeline : only English sample

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,ComplementNB
#lightGBM
import optuna
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix,hstack

In [ ]:
def proba_val(model,xless,xmore):
    p1 = model.predict_proba(xless)[:,1]
    p2 = model.predict_proba(xmore)[:,1]
    return (p1<p2).mean()

In [ ]:
tf1 = TfidfVectorizer()
mb = MultinomialNB()

In [ ]:
X1 = tf1.fit_transform(en['comment_text'])
y1 = en['y']

In [ ]:
X1_p = csr_matrix(en.loc[:,ft_cols].to_numpy())
X1_p.shape,X1.shape

In [ ]:
X1_all = hstack((X1,X1_p))

In [ ]:
v1 = tf1.transform(less['comment_text'])
v2 = tf1.transform(more['comment_text'])

In [ ]:
v1_p = csr_matrix(less.loc[:,ft_cols].to_numpy())
v2_p = csr_matrix(more.loc[:,ft_cols].to_numpy())
v1_all = hstack((v1,v1_p))
v2_all = hstack((v2,v2_p))

In [ ]:
mb.fit(X1,y1)

In [ ]:
proba_val(mb,v1,v2)

In [ ]:
# use extended feature but reduce correct rate
mb2 = MultinomialNB()
mb2.fit(X1_all,y1)
proba_val(mb2,v1_all,v2_all)

## ridge

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
def eval_ridge(model,v1,v2):
    p1 = model.predict(v1)
    p2 = model.predict(v2)
    # Validation Accuracy
    return (p1 < p2).mean()

In [ ]:
ridge1 = Ridge(alpha=1)
ridge1.fit(X1,en['ys'])
print('successfully fit')
print(eval_ridge(ridge1,v1=v1,v2=v2))
print('')

In [ ]:
ridge2 = Ridge(alpha=0.5)
ridge2.fit(X1,en['ys'])
print('successfully fit')
print(eval_ridge(ridge2,v1=v1,v2=v2))
print('')

In [ ]:
ridge3 = Ridge(alpha=2)
ridge3.fit(X1,en['ys'])
print('successfully fit')
print(eval_ridge(ridge3,v1=v1,v2=v2))
print('')

In [ ]:
ridge4 = Ridge(alpha=2.5) #alpha=3, correct rate=0.6872
ridge4.fit(X1,en['ys'])
print('successfully fit')
print(eval_ridge(ridge4,v1=v1,v2=v2))
print('')

## augment ridge

In [ ]:
tf1es = TfidfVectorizer()
X1es = tf1es.fit_transform(es['comment_text'])
yses = es['ys']

In [ ]:
tf1fr = TfidfVectorizer()
X1fr = tf1fr.fit_transform(fr['comment_text'])
ysfr = fr['ys']

In [ ]:
tf1de = TfidfVectorizer()
X1de = tf1de.fit_transform(de['comment_text'])
ysde = de['ys']

In [ ]:
v1de=tf1de.transform(less['comment_text'])
v2de=tf1de.transform(more['comment_text'])
v1fr=tf1fr.transform(less['comment_text'])
v2fr=tf1fr.transform(more['comment_text'])
v1es=tf1es.transform(less['comment_text'])
v2es=tf1es.transform(more['comment_text'])

In [ ]:
ridge1de = Ridge(alpha=1)
ridge1de.fit(X1de,de['ys'])
print('successfully fit')
print(eval_ridge(ridge1de,
                 v1=v1de,
                 v2=v2de))

In [ ]:
ridge1fr = Ridge(alpha=1)
ridge1fr.fit(X1fr,fr['ys'])
print('successfully fit')
print(eval_ridge(ridge1fr,
                 v1=v1fr,
                 v2=v2fr))

In [ ]:
ridge1es = Ridge(alpha=1)
ridge1es.fit(X1es,es['ys'])
print('successfully fit')
print(eval_ridge(ridge1es,
                 v1=v1es,
                 v2=v2es))

In [ ]:
ridge2de = Ridge(alpha=0.5)
ridge2de.fit(X1de,de['ys'])
print('successfully fit')
print(eval_ridge(ridge2de,
                 v1=v1de,
                 v2=v2de))

ridge2fr = Ridge(alpha=0.5)
ridge2fr.fit(X1fr,fr['ys'])
print('successfully fit')
print(eval_ridge(ridge2fr,
                 v1=v1fr,
                 v2=v2fr))

ridge2es = Ridge(alpha=0.5)
ridge2es.fit(X1es,es['ys'])
print('successfully fit')
print(eval_ridge(ridge2es,
                 v1=v1es,
                 v2=v2es))

In [ ]:
ridge3de = Ridge(alpha=2)
ridge3de.fit(X1de,de['ys'])
print('successfully fit')
print(eval_ridge(ridge3de,
                 v1=v1de,
                 v2=v2de))

ridge3fr = Ridge(alpha=2)
ridge3fr.fit(X1fr,fr['ys'])
print('successfully fit')
print(eval_ridge(ridge3fr,
                 v1=v1fr,
                 v2=v2fr))

ridge3es = Ridge(alpha=2)
ridge3es.fit(X1es,es['ys'])
print('successfully fit')
print(eval_ridge(ridge3es,
                 v1=v1es,
                 v2=v2es))

In [ ]:
#average of four language
p1 = ridge1.predict(v1)+ridge1es.predict(v1es)+ridge1fr.predict(v1fr)+ridge1de.predict(v1de)
p2 = ridge1.predict(v2)+ridge1es.predict(v2es)+ridge1fr.predict(v2fr)+ridge1de.predict(v2de)
# Validation Accuracy
(p1 < p2).mean()

In [ ]:
#average of four language
p1 = ridge2.predict(v1)+ridge2es.predict(v1es)+ridge2fr.predict(v1fr)+ridge2de.predict(v1de)
p2 = ridge2.predict(v2)+ridge2es.predict(v2es)+ridge2fr.predict(v2fr)+ridge2de.predict(v2de)
# Validation Accuracy
(p1 < p2).mean()

In [ ]:
p1 = ridge3.predict(v1)+ridge3es.predict(v1es)+ridge3fr.predict(v1fr)+ridge3de.predict(v1de)
p2 = ridge3.predict(v2)+ridge3es.predict(v2es)+ridge3fr.predict(v2fr)+ridge3de.predict(v2de)
# Validation Accuracy
(p1 < p2).mean()

## only extend positive label

In [ ]:
#import random
#random.seed(2333)
#len(id0),len(id1)
#sample_index = random.sample(range(len(id0)),len(id1))
#id0_s = id0.iloc[list(sample_index)]
#sample_id = [id1,id0_s]

In [ ]:
#extended dataset
ext = pd.concat([en,
es[es['y']==1],
fr[fr['y']==1],
de[de['y']==1]])

In [ ]:
tf1ext = TfidfVectorizer()
X1ext = tf1ext.fit_transform(ext['comment_text'])
ysext = ext['ys']

In [ ]:
v1ext=tf1ext.transform(less['comment_text'])
v2ext=tf1ext.transform(more['comment_text'])

In [ ]:
ridge1ext = Ridge(alpha=1)
ridge1ext.fit(X1ext,ext['ys'])
print('successfully fit')
print(eval_ridge(ridge1ext,
                 v1=v1ext,
                 v2=v2ext))

ridge2ext = Ridge(alpha=0.5)
ridge2ext.fit(X1ext,ext['ys'])
print('successfully fit')
print(eval_ridge(ridge2ext,
                 v1=v1ext,
                 v2=v2ext))

ridge3ext = Ridge(alpha=2)
ridge3ext.fit(X1ext,ext['ys'])
print('succextsfully fit')
print(eval_ridge(ridge3ext,
                 v1=v1ext,
                 v2=v2ext))

In [ ]:
#acc2 = mean_squared_error(y2, model2.predict(X2))/ mean_squared_error(y2,np.repeat(np.mean(y2),len(y2)))

## use different machine and take average

In [ ]:
#average of four language
p1 = ridge1.predict(v1)+ridge1es.predict(v1es)+ridge1fr.predict(v1fr)+ridge1de.predict(v1de)+ridge1ext.predict(v1ext)
p2 = ridge1.predict(v2)+ridge1es.predict(v2es)+ridge1fr.predict(v2fr)+ridge1de.predict(v2de)+ridge1ext.predict(v2ext)
# Validation Accuracy
(p1 < p2).mean()

In [ ]:
#average of four language
p1 = ridge2.predict(v1)+ridge2es.predict(v1es)+ridge2fr.predict(v1fr)+ridge2de.predict(v1de)+ridge2ext.predict(v1ext)
p2 = ridge2.predict(v2)+ridge2es.predict(v2es)+ridge2fr.predict(v2fr)+ridge2de.predict(v2de)+ridge2ext.predict(v2ext)
# Validation Accuracy
(p1 < p2).mean()

In [ ]:
#average of four language
p1 = ridge3.predict(v1)+ridge3es.predict(v1es)+ridge3fr.predict(v1fr)+ridge3de.predict(v1de)+ridge3ext.predict(v1ext)
p2 = ridge3.predict(v2)+ridge3es.predict(v2es)+ridge3fr.predict(v2fr)+ridge3de.predict(v2de)+ridge3ext.predict(v2ext)
# Validation Accuracy
(p1 < p2).mean()

## submission

In [ ]:
df_sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
df_sub.head()

In [ ]:
X_test1 =tf1.transform(df_sub['text'])
X_testde=tf1de.transform(df_sub['text'])
X_testfr=tf1fr.transform(df_sub['text'])
X_testes=tf1es.transform(df_sub['text'])

In [ ]:
X_testext=tf1ext.transform(df_sub['text'])

In [ ]:
p3 = 3*ridge3.predict(X_test1)+ridge3es.predict(X_testes)+ridge3fr.predict(X_testfr)+ridge3de.predict(X_testde)+ridge3ext.predict(X_testext)

In [ ]:
df_sub['score'] = p3

In [ ]:
df_sub['score'].count()

In [ ]:
df_sub['score'].count()-df_sub['score'].nunique()

In [ ]:
df_sub[['comment_id', 'score']].to_csv('submission.csv', index=False)

In [ ]:
df_sub.head()

In [ ]:
df_sub[['comment_id', 'score']].to_csv('submission.csv', index=False)